### Beginning

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import tensorflow as tf
import numpy as np
import h5py, os, datetime

In [0]:
hdf_dir = "/content/drive/My Drive/DL/DIV2K HDF/"

def get_hdf_dir(training=True, hsv=False):
    if hsv == False: cmodel = "RGB/"
    else: cmodel = "HSV/"
    if training == True: data = "train/"
    else: data = "valid/"
    return os.path.join(hdf_dir, cmodel, data)
  
def gen_all_images(training, hsv):
    if training == True: total = 800
    else: total = 100
    dir_ = get_hdf_dir(training, hsv)
    for i in range(total):
        img = dir_ + "img{}.hdf5".format(i + 1)
        with h5py.File(img, "r") as hdf:
            x = hdf["lr_x2_img{}".format(i + 1)][()]
            y = hdf["hr_img{}".format(i + 1)][()]
            size = hdf["patched_res_img{}".format(i + 1)][()]
            number = i
        yield x, y, size, number
        
def save_img(img, filename, hsv=False):
    if hsv == True:
        img = tf.image.hsv_to_rgb(img)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    img_raw = tf.image.encode_png(img).eval()
    return tf.write_file(tf.constant(filename), img_raw) 
     
def image_reconstructing(img, img_size):
    new_img = []
    for i in range(img_size[0]):
        k, l = i * img_size[1], (i + 1) * img_size[1]
        new_line = np.concatenate((img[k:l]), axis=1)
        new_img.append(new_line)
    new_img = np.asarray(new_img)
    new_img_2 = np.concatenate((new_img[:]), axis=0)
    return new_img_2

### Architecture

In [0]:
def conv_layer(x, W_shape, b_shape):
    W = tf.get_variable("W", shape=W_shape, initializer=tf.contrib.layers.xavier_initializer_conv2d())
    b = tf.get_variable("b", initializer=tf.constant(0.01, shape=b_shape))
    conv = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")
    conv_b = tf.nn.bias_add(conv, b)
    return conv_b

def res_block(x, scope):
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE) as scope:
        shortcut = x
        with tf.variable_scope("conv1", reuse=tf.AUTO_REUSE) as scope:
            x = conv_layer(x, [5,5,32,32], [32])
        x = tf.nn.relu(x)
        with tf.variable_scope("conv2", reuse=tf.AUTO_REUSE) as scope:
            x = conv_layer(x, [5,5,32,32], [32])
        x = x * 0.1  
    return x + shortcut

In [0]:
def net(x):
    #First Conv
    with tf.variable_scope("first_conv", reuse=tf.AUTO_REUSE) as scope:
        conv1 = conv_layer(x, [5,5,3,32], [32])    
    shortcut1 = conv1
    #ResBlocks Stack
    res_stack = tf.contrib.layers.repeat(conv1, 16, res_block, scope="res_stack")
    #Conv after ResBlocks
    with tf.variable_scope("after_res_block", reuse=tf.AUTO_REUSE) as scope:
        conv2 = conv_layer(res_stack, [5,5,32,32], [32])
    #Shortcut before Shuffling
    res1 = 0.1 * conv2 + shortcut1
    #Conv befor Shuffling
    with tf.variable_scope("before_shuffling", reuse=tf.AUTO_REUSE) as scope:
        conv3 = conv_layer(res1, [5,5,32,64], [64])
    #Shuffling
    shuffled = tf.nn.depth_to_space(conv3, 2, data_format='NHWC') #maybe NCWH is better?
    #Conv to 3 channels
    with tf.variable_scope("last_conv", reuse=tf.AUTO_REUSE) as scope:
        conv4 = conv_layer(shuffled, [5,5,16,3], [3])
    return conv4

In [0]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 24, 24, 3])
y = tf.placeholder(tf.float32, [None, 48, 48, 3])
y_net = net(x)

### Loss

In [0]:
vars_ = [var for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)]
regularization = tf.multiply(tf.reduce_sum([tf.nn.l2_loss(var) for var in vars_]), 0.01)

learning_rate = tf.placeholder(tf.float32)
mae = tf.reduce_mean(tf.abs(tf.subtract(y_net, y)))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(tf.add(mae, regularization))

### Training

In [0]:
TRAINING = True
HSV = True
BATCH = 32
EPOCHS = 15

if HSV == False: save_path = "/content/drive/My Drive/DL/ckpt/SR_RGB_"
else: save_path = "/content/drive/My Drive/DL/ckpt/SR_HSV_"
  
print("Start:", datetime.datetime.now().strftime("%m/%d %H:%M:%S"))
  
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=30)
    for i in range(EPOCHS):
        if i <= 5: lr = 0.00001
        else: lr = 0.000001
        for lr_img, hr_img, size, number in gen_all_images(training=TRAINING, hsv=HSV):
            batch_count = (size[0] * size[1]) // BATCH
            for j in range(batch_count):
                k, l = (BATCH * j), BATCH * (j + 1) - 1
                batch_x, batch_y = lr_img[k:l], hr_img[k:l]
                _, loss = sess.run([ train_step, mae ], feed_dict={ x: batch_x, y: batch_y, learning_rate: lr })
                if number % 100 == 0 and j == 0:
                    print("time: {}... epoch {}... image {}... loss = {:.7f}".format(datetime.datetime.now().strftime("%m/%d %H:%M:%S"), i, number, loss))
        saver.save(sess, save_path + "{}.ckpt".format(i))      
    
print("End:", datetime.datetime.now().strftime("%m/%d %H:%M:%S"))    

### Result

In [0]:
TRAINING = False
HSV = False
print("Choose your epoch (0-14):")
EPOCH = input()
test_images = [2, 8, 9, 11, 15, 28, 37, 41, 52, 55, 98]

if HSV == False: 
    ckpt_path = "/content/drive/My Drive/DL/ckpt/SR_RGB_"
    metrics_path = "/content/drive/My Drive/DL/metrics/RGB_SR_PSNR_SSIM_"
    img_save_path = "/content/drive/My Drive/DL/test_img/RGB/"
else: 
    ckpt_path = "/content/drive/My Drive/DL/ckpt/SR_HSV_"
    metrics_path = "/content/drive/My Drive/DL/metrics/HSV_SR_PSNR_SSIM_"
    img_save_path = "/content/drive/My Drive/DL/test_img/HSV/"
    
print("Start:", datetime.datetime.now().strftime("%m/%d %H:%M:%S"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, "{}{}.ckpt".format(ckpt_path, EPOCH))
    get_upsc_img = net(x)
    metrics = []
    for lr_img, hr_img, size, number in gen_all_images(training=TRAINING, hsv=HSV):
        if number in test_images:
            upsc_img = sess.run(get_upsc_img, feed_dict={ x : lr_img })
            new_img, old_img = image_reconstructing(np.asarray(upsc_img), size), image_reconstructing(np.asarray(hr_img), size)
            new_img_t, old_img_t = tf.convert_to_tensor(new_img), tf.convert_to_tensor(old_img)
            ssim, psnr = tf.image.ssim(new_img_t, old_img_t, max_val=1.0).eval(), tf.image.psnr(new_img_t, old_img_t, max_val=1.0).eval()
            metrics.append((ssim, psnr))
            print("time: {}... image {}".format(datetime.datetime.now().strftime("%m/%d %H:%M:%S"), number))
            img_saving = save_img(new_img, "{}img{}_epoch{}.png".format(img_save_path, number, EPOCH), hsv=HSV)
            sess.run(img_saving)
    metrics = np.asarray(metrics)
    print(metrics.shape)
    np.save("{}epoch_{}".format(metrics_path, EPOCH), metrics)
                 
print("End:", datetime.datetime.now().strftime("%m/%d %H:%M:%S"))